In [2]:
from tkinter import *
import tkinter as tk
from PIL import Image
from PIL import ImageTk
from tkinter import filedialog
import cv2
import math
import numpy as np

In [23]:
WIDTH, HEIGHT = 600, 600
topx, topy, botx, boty = 0, 0, 0, 0
rect_id = None
label = None
name = ""
def loading(): #load image
    global name
    path = filedialog.askopenfilename(filetypes = [("TIFF", "*.tif"), ("All Files", "*.*")], title = "Choose File")
    name = path
    if len(path) >0:
        return cv2.imread(path)
    else:
        exit(0)
def toTkImg(img):
    tkimg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    tkimg = Image.fromarray(tkimg)
    tkimg = ImageTk.PhotoImage(tkimg)
    return tkimg
    
def get_mouse_posn(event):
    global topy, topx
    topx, topy = event.x, event.y

def update_sel_rect(event):
    global rect_id
    global topy, topx, botx, boty
    botx, boty = event.x, event.y
    canvas.coords(rect_id, topx, topy, botx, boty)  # Update selection rect.

def calculate(roi): #Find length and width bacterial cell in ROI
    ret, thresh = cv2.threshold(roi,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU) #adaptive thresholding with Otsu method
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 1:
        ellipse = cv2.fitEllipse(contours[0])
        majoraxis = max(ellipse[1])
        minoraxis = min(ellipse[1])
        
        a = majoraxis/2
        b = minoraxis/2
        area = a*b*math.pi
        return [round(majoraxis,3), round(minoraxis,3), round(area,3)] #returns length of major axis and area of cell
    elif len(contours) > 1:
        cell = 0
        for i in range(len(contours)):
            if len(contours[i]) > len(contours[cell]):
                cell = i
        ellipse = cv2.fitEllipse(contours[cell])
        majoraxis = max(ellipse[1])
        minoraxis = min(ellipse[1])
        
        a = majoraxis/2
        b = minoraxis/2
        area = a*b*math.pi
        return [round(majoraxis,3), round(minoraxis,3), round(area,3)] #returns length of major axis and area of cell

    
    
def update_text(event):
    global imgray
    global rect_id
    global label
    if rect_id != None:
        #if the rectangle is created
        #get the region of interest where y is row and x is col
        selection = imgray[min(topy, boty):max(topy, boty), min(topx,botx):max(topx,botx)] 
        selection = process(selection)
        value = calculate(selection)
        if label == None:
            label = Label(window, text = "Length: %f \tWidth: %f \n \nArea: %f" % (value[0],value[1], value[2]), pady = 10, padx = 10, relief = "solid")
            label.pack(side = TOP, fill = X, expand = True)
        else:
            label.configure(text = "Length: %f \tWidth: %f \nArea: %f" % (value[0],value[1], value[2]))

def process(selection):
    #noise = cv2.fastNlMeansDenoising(selection, None, 37) #noise reduction
    #norm = np.zeros(selection.shape)
    #normalized = cv2.normalize(noise,  norm, 0, 255, cv2.NORM_MINMAX) #normalization/enhancement image
    gauss = cv2.GaussianBlur(selection,(5,5),cv2.BORDER_DEFAULT) #gaussian blurring
    #canny = cv2.Canny(gauss, 0,0) #create canny edge detection for fit ellipse
    return gauss

window = tk.Tk()
window.geometry('%sx%s' % (WIDTH, HEIGHT))
window.configure(background='grey')

cv2img = loading()
window.title(name)
imgray = cv2.cvtColor(cv2img, cv2.COLOR_BGR2GRAY) #creates grayscale and gaussian blur image to use later



tkimg = toTkImg(cv2img)
canvas = tk.Canvas(window, width=tkimg.width(), height=tkimg.height(),
                   borderwidth=0, highlightthickness=0)
canvas.pack(expand=True)
canvas.img = tkimg  # Keep reference in case this code is put into a function.
canvas.create_image(0, 0, image=tkimg, anchor=tk.NW)



# Create selection rectangle (invisible since corner points are equal).
rect_id = canvas.create_rectangle(topx, topy, topx, topy,
                                  dash=(2,2), fill='', outline='white')
    
canvas.bind('<Button-1>', get_mouse_posn)
canvas.bind('<B1-Motion>', update_sel_rect)
canvas.bind('<ButtonRelease-1>', update_text) 

window.mainloop()